# **Coarse-Grained Mapping of DNA**

This Jupyter notebook maps full atomistic trajectories to CG trajectories ....

---
---
# **Import of the necesary packages for CG-mapping**

Firstly, we need to install all necessary libraries and packages for our mapping. The main packages are:

1.    mdtraj (https://amber-md.github.io/pytraj/latest/index.html)
2.    MDAnalysis (https://pypi.org/project/py3Dmol/)
3.    Numpy (https://numpy.org/)

In [ ]:
#@title **Load dependencies**
import sys
from importlib.machinery import SourceFileLoader
import os
import numpy as np
import mdtraj as md
import MDAnalysis as mda
from MDAnalysis import transformations
from __future__ import print_function
from tqdm import tqdm

import src.cgmap as cg
workDir = os.getcwd()
parentDir = os.path.abspath(os.path.join(workDir, os.pardir))
# sys.path.append(os.path.join(parentDir, 'src'))
# cg = SourceFileLoader('cgmap', os.path.join(parentDir, 'src/cgmap.py')).load_module()

---
---
# **Loading the necessary input files**

Below, you should provide the names of all input files and the pathway of your folder containing them.


In [ ]:
#@title Provide the necessary input files below: 
#@markdown 
Top_file_name = 'Data/DNA-1/MD.gro' #@param {type:"string"}
Traj_file_name = 'Data/DNA-1/MD.trr' #@param {type:"string"}
Top_file = os.path.join(workDir, str(Top_file_name))
Traj_file = os.path.join(workDir, str(Traj_file_name))

u = mda.Universe(Top_file, Traj_file)

In [ ]:
# Optional code to wrap the trajectory
ag = u.atoms
transform = transformations.wrap(ag)
u.trajectory.add_transformations(transform)

ag.write('DNA1_wrap.dcd', frames='all')
Traj_file = os.path.join(workDir, 'DNA1_wrap.dcd')

In [ ]:
trj = md.load(Traj_file,top=Top_file)

# **Coarse-Grained Mapping**

## Coordinates mapping

In [ ]:
basepairs_per_bead = 0  # 0 means 1 nucleotide per bead
Output_PDB_file = 'Data/DNA1_CG.pdb' 
Output_Traj_file = 'Data/DNA1_CG.dcd' 
Output_PDB = os.path.join(workDir, str(Output_PDB_file))
Output_Traj = os.path.join(workDir, str(Output_Traj_file))

# Map the DNA to CG beads
maps = cg.DNA_CG_map(basepairs_per_bead)
    
if basepairs_per_bead == 0:
    DNA_residues = trj.top.select("(resid 0 to 39)").residues.resnames
    
else:
    label_list = ["DNA"]*int(20/basepairs_per_bead)

index_list = []
for sel in maps:
  bead_indices = trj.top.select(sel)
  index_list.append(bead_indices)

cg_trj = cg.cg_by_index(trj, index_list, label_list)
cg_trj.save(Output_Traj) 
cg_trj[0].save(Output_PDB)
with open('data/DNA1_CGxyz.npy', 'wb') as f:
   np.save(f,cg_trj.xyz)

## Forces mapping

In [ ]:
numbeads = len(index_list)
dna=u.select_atoms("nucleic")
pos = dna.positions
print(pos[-1])

forces_cg = []
forces_cg_i = []
forces_cg_frame = []
for t in tqdm(u.trajectory):
    forces_frame = dna.forces
    for i in range(numbeads):
        bead_indexes = index_list[i]
        forces_cg_i = cg.map_forces(forces_frame,bead_indexes) # Check map_forces function is correct
        forces_cg_frame.append(np.array(forces_cg_i))
    forces_cg.append(np.array(forces_cg_frame))
    
forces_cg_np = np.array(forces_cg)
print(forces_cg_np.shape)
np.save('Data/DNA1_CGfrc.npy',forces_cg_np)

## Embedding

In [ ]:
print(DNA_residues)

In [ ]:
Bead2INT = {'DA':1,
            'DT':2,
            'DG':3,
            'DC':4
            }

emb = np.array([Bead2INT[x] for x in DNA_residues])
np.save('DNA1_embeddings.npy', emb)

embeddings = np.tile(emb, [forces_cg_np.shape[0], 1])
print("Embeddings size: {}".format(embeddings.shape))